In [5]:
'''this notebook is version 1 of the data pipeline responsible for collecting and preparing 
new images from users that have taken a picture using the app'''

# Importing necessary libraries:
import boto3
import datetime
import psycopg2
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

False

In [3]:
def upload_user_image(image_data):
    ''' Retreive picture from app request and store it in s3 '''

    # Initialize the S3 client:
    s3 = boto3.client('s3')

    # Declare bucket file variables:
    folder_name = 'raw_data/nonlabelled_images/date=27-03-2024/'
    s3_bucket = 'provafinalproject-test'

    # Generating a unique filename (We can use any method we prefer while keeping in mind GDPR)
    current_datetime = datetime.now().strftime("%Y%m%d%H%M%S")
    user_tag = 'user1' #somehow assign a user variable?
    filename = f"{current_datetime}_{user_tag}.jpg"  # Example: 20240428153000_user1.jpg

    # Combine folder and filename to create the key
    key = folder_name + filename

    # Upload the image data to S3:
    try:
        s3.upload_fileobj(image_data, s3_bucket, key)
        return f"Successfully uploaded image to S3: {key}"
    except Exception as e:
        return f"Error uploading image to S3: {e}"

In [6]:
def store_newimage_metadata(image_data, user_id, filename, skin_tone, key):
    ''' This function stores metadata from the new image to indicate that
    it has not been officially labeled by a specialist after diagnosis'''
    try:
        # Connect to the PostgreSQL database using environment variables:
        connection = psycopg2.connect(
            dbname=os.getenv("DB_NAME"),
            user=os.getenv("DB_USER"),
            password=os.getenv("DB_PASSWORD"),
            host=os.getenv("DB_HOST"),
            port=os.getenv("DB_PORT")
        )

        # Create a cursor object to execute SQL queries:
        cursor = connection.cursor()

        # Create a new table named 'new_image_metadata' if it doesn't exist:
        create_table_query = '''
        CREATE TABLE IF NOT EXISTS new_image_metadata (
            user_id SERIAL PRIMARY KEY,
            filename VARCHAR(255),
            skin_tone INTEGER,
            malignant VARCHAR(10) DEFAULT 'TBD',
            path VARCHAR(255),
            time_taken TIMESTAMP
        )
        '''
        cursor.execute(create_table_query)
        connection.commit()

        # Get current timestamp
        timestamp = datetime.datetime.now()

        # Insert image metadata into the table:
        insert_query = '''
        INSERT INTO new_image_metadata (filename, skin_tone, malignant, path, time_taken)
        VALUES (%s, %s, %s, %s, %s)
        '''
        cursor.execute(insert_query, (filename, 'TBD', 'TBD', key, timestamp)) # skin_tone value gets verified by specialist as well
        connection.commit()

        print("Image metadata inserted successfully.")

    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL or executing queries:", error)

    finally:
        # Close the cursor and connection:
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed.")


In [4]:
def retreive_for_review(user_id):
    '''This function gets the image from the S3 bucket so that the specialist can 
    update it with the final diagnosis'''

    # Initiate boto 3 client:
    s3 = boto3.client('s3')

    # Declare bucket file variables:
    folder_name = 'raw_data/nonlabelled_images/date=27-03-2024/'
    s3_bucket = 'provafinalproject-test'

    return "OK"
    

In [ ]:
def add_final_diagnosis():
    '''This function labels the metadata for the user image with the final
    diagnosis performed by the specialist after consultation.'''

    return "OK"

In [ ]:
def add_final_skintype():
    '''This function labels the metadata for the user skintype with the final
    diagnosis performed by the specialist after consultation.'''

    return "OK"

In [7]:
def add_to_merged_dataset():
    '''This function adds the labelled image to the merged dataset for modeling
     and sends the image to the labelled S3 bucklet...'''
    variable = myvar

    return "OK"

In [ ]:
# Trigger message for patient: Doctor needs more images
# Loop of feedback for doctor
# Nivel de confianza del diagnostico del modelo... que el médico tenga un mecanismo de feedback para decir que tambien lo hizo